# Mini Lab 1
Group 2 
Members: Josh Mitchell, Adam Alidra, Ryan Herrin

### Requirements
- Create a logistic regression model and a support vector machine model for the classification task involved with your dataset. Assess how well each model performs (use 80/20 training/testing split for your data). Adjust parameters of the models to make them more accurate. If your dataset size requires the use of stochastic gradient descent, then linear kernel only is fine to use. That is, the SGDClassifier is fine to use for optimizing logistic regression and linear support vector machines. For many problems, SGD will be required in order to train the SVM model in a reasonable timeframe.
- Discuss the advantages of each model for each classification task. Does one type of model offer superior performance over another in terms of prediction accuracy? In terms of training time or efficiency? Explain in detail.
- Use the weights from logistic regression to interpret the importance of different features for the classification task. Explain your interpretation in detail. Why do you think some variables are more important?
- Look at the chosen support vectors for the classification task. Do these provide any insight into the data? Explain. If you used stochastic gradient descent (and therefore did not explicitly solve for support vectors), try subsampling your data to train the SVC model— then analyze the support vectors from the subsampled dataset.

## Setup 

In [11]:
# Libraries
import numpy as np
import pandas as pd
from sklearn import metrics as mt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [8]:
# Read in data 

# NOTE: Data imported has been generated from the first lab and turned into two data sets 
# + One data set is the raw data minus columns that had no values in them 
# + The other data set is data that had a transformation applied to it 
raw_data_location = '../dataset/raw_data.csv'
transformed_data_location = '../dataset/transformed_data.csv'

raw_data_df = pd.read_csv(raw_data_location)
tf_data_df = pd.read_csv(transformed_data_location) # tf being shorthand for transformed


## Logistic Regression Model and SVM Model

#### Split data into 80/20 Train Test 

In [10]:
# Define common 
seed_value = 8675309 # if you know you know
test_sample_size = .2 # percentage 

# Split the data into a 80/20 Train/Testing for raw and trasnformed data 
# Using the split method from sklearn 
raw_train, raw_test = train_test_split(raw_data_df, test_size=test_sample_size, random_state=seed_value)
tf_train, tf_test = train_test_split(tf_data_df, test_size=test_sample_size, random_state=seed_value)

### Logistic Regression Model

In [ ]:
# Logistic Regression Model for Raw Data 


In [ ]:
# Logistic Regression Model for Transformed Data 

### SVM Model

In [ ]:
# SVM Model for Raw Data

In [ ]:
# SVM Model for Transformed Data